- Windows10 64bit
- Python3.5
- MongoDB 4.0.2

参考

- [『Pythonクローリング＆スクレイピング―データ収集・解析のための実践開発ガイド―』技術評論社](https://gihyo.jp/book/2017/978-4-7741-8367-1)
- [『退屈なことはPythonにやらせよう――ノンプログラマーにもできる自動化処理プログラミング』O'Reilly Japan](https://www.oreilly.co.jp/books/9784873117782/)

# PythonからMongoDBを使用

MongoDB用のディレクトリを作成しておく

バックグラウンドでMongoDBを起動しておく

起動

`mongod --dbpath <MongoDBのデータ用ディレクトリへのパス>`


参考：http://kageura.hatenadiary.jp/entry/2018/01/09/Windows%E7%89%88MongoDB%E3%81%AE%E3%82%A4%E3%83%B3%E3%82%B9%E3%83%88%E3%83%BC%E3%83%AB%E3%83%BBMongoShell%E3%82%92%E9%80%9A%E3%81%97%E3%81%A6CRUD%E3%82%B3%E3%83%9E%E3%83%B3%E3%83%89%E3%82%92%E6%89%93

`BulkWriteError: batch op errors occurred`

でよく止まる

→動画が存在しない部分まで検索対象としてしまっている可能性がある。

In [ ]:
import os
import sys

from apiclient.discovery import build # pip install google-api-python-client
from pymongo import MongoClient, DESCENDING

YOUTUBE_API_KEY = '<APIキーを記入>' # 環境変数（.envファイル）からAPIを呼び出したいが、JupyterNotebookからだとなぜかエラーになる。

query = '輝夜月'
max_pages=8
maxResults=50

def main():
    """
    メイン処理
    """
    mongo_client = MongoClient('localhost', 27017)
    db = mongo_client.kaguya2
    collection = db.luna # kaguyaデータベース -> lunaコレクション
    collection.delete_many({}) # 既存の全てのドキュメントを削除

    sort_key='statistics.viewCount' # ランキング基準の指定

    for items_per_page in search_videos(query, max_pages, maxResults):
        save_to_mongodb(collection, items_per_page)


def search_videos(query, max_pages=10, maxResults=50):
    """
    動画を検索してページ単位でlistをyieldする
    """
    youtube = build('youtube', 'v3', developerKey = YOUTUBE_API_KEY)

    search_request = youtube.search().list(
        part='id',
        q=query,
        type='video',
        maxResults=maxResults,
    )
    
    i = 0
    while search_request and i < max_pages:
        search_response = search_request.execute() # execute()で実際にHTTPリクエストを送信。APIのレスポンスを取得。
        video_ids = [item['id']['videoId'] for item in search_response['items']]

        videos_response = youtube.videos().list(
            part='snippet,statistics',
            id=','.join(video_ids)
        ).execute()

        yield videos_response['items']

        search_request = youtube.search().list_next(search_request, search_response)
        i += 1


def save_to_mongodb(collection, items):
    """
    MongoDBにアイテムのリストを保存
    """
    for item in items:
        item['_id'] = item['id']

        for key, value in item['statistics'].items():
            item['statistics'][key] = int(value)

    result = collection.insert_many(items) # コレクションに挿入
    print('Inserted {0} documents'.format(len(result.inserted_ids)), file=sys.stderr)


if __name__ == '__main__':
    main()


# 整形してMongoDBから抽出したデータをcsvに保存 

タイトル、URL、チャンネル名、公開日、視聴回数、Like数、Dislike数、コメント数（一部不正確？）、favorite数（不明）

In [ ]:
import csv

mongo_client = MongoClient('localhost', 27017)
db = mongo_client.kaguya
collection = db.luna # kaguyaデータベース -> lunaコレクション

# 列名（1行目）を作成
## [タイトル、URL、チャンネル名、公開日、視聴回数、Like数、Dislike数、コメント数（一部不正確？）、favorite数（不明）]
col_name = ['title', 'url', 'channelTitle', 'publishedAt']
statistics_keys = ['viewCount', 'likeCount', 'dislikeCount', 'commentCount', 'favoriteCount']
col_name.extend(statistics_keys)
sort_key = 'statistics.likeCount' # ひとまずLikeが多い順に保存

with open('youtube_result.csv', 'w', newline='', encoding='utf-8') as output_csv:
    csv_writer = csv.writer(output_csv)
    csv_writer.writerow(col_name) # 列名を記入
    
    # データを整形しつつcsvに書き込んでいく
    for item in collection.find().sort(sort_key, DESCENDING):
        url = 'https://www.youtube.com/watch?v=' + item['_id']
        row_items = [item['snippet']['title'], url, item['snippet']['channelTitle'], item['snippet']['publishedAt']]
        
        # 値が入っていない部分を埋めるために統計量についてfor文を回す
        for statistics_key in statistics_keys:
            item['statistics'].setdefault(statistics_key, 0)
            row_items.append(item['statistics'][statistics_key]) # 列名と同じ順番になるようにstatistics_keysでfor文を回す

        csv_writer.writerow(row_items)


In [ ]:
# 輝夜月の公式動画のみ抽出
for item in collection.find({'snippet.channelTitle': 'Kaguya Luna Official'}).sort('statistics.viewCount', DESCENDING):
    print(item['statistics']['viewCount'], item['snippet']['channelTitle'], item['snippet']['title'])

# 保存したデータをpandasに読み込んで分析

In [ ]:
import pandas as pd

# csvファイルを読み込む
df = pd.read_csv('youtube_result.csv')

# 「活発度」列を作る
# 再生数に対してどれだけのコメントが集まっているか
df['活発度'] = df['commentCount'] / df['viewCount']

# 「好感度」列を作る
# 再生数に対してどれだけLikeが集まっているか
df['好感度'] = df['likeCount'] / df['viewCount']

In [ ]:
# 好感度順にソート
# オフィシャル動画のみ抽出
df[df['channelTitle'] == 'Kaguya Luna Official'].sort_values(by=['好感度'], ascending=False)

参考

In [ ]:
# 視聴回数上位100位で活発度順にソート
df.sort_values(by=['viewCount'], ascending=False).head(100).sort_values(by=['活発度'], ascending=False)